### Task 2: Change Point Modeling and Insight Generation

#### Part 2.1: Core Analysis Implementation

In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import pytensor.tensor as pt

def bayesian_change_point_analysis(df, max_changepoints=5):
    """
    Perform Bayesian change point detection on oil price data
    """
    prices = df['Price'].values
    n_obs = len(prices)
    
    with pm.Model() as model:
        # Priors for change points (uniform over time)
        tau = pm.Uniform("tau", lower=0, upper=n_obs, shape=max_changepoints)
        
        # Sort change points chronologically
        sorted_tau = pm.Deterministic("sorted_tau", pt.sort(tau))
        
        # Segment parameters
        means = pm.Normal("means", mu=0, sigma=10, shape=max_changepoints+1)
        sigma = pm.HalfNormal("sigma", sigma=1)
        
        # Determine regime for each observation
        regime = np.zeros(n_obs, dtype=int)
        for i in range(max_changepoints):
            regime = regime + (np.arange(n_obs) >= sorted_tau[i])
        
        # Likelihood
        pm.Normal(
            "likelihood", 
            mu=means[regime], 
            sigma=sigma, 
            observed=prices
        )
        
        # Sampling
        trace = pm.sample(
            draws=2000,
            tune=1000,
            chains=2,
            target_accept=0.9,
            return_inferencedata=True
        )
    
    return trace

def analyze_results(trace, df):
    """Analyze and visualize change point results"""
    # Extract change points
    tau_samples = trace.posterior["sorted_tau"].values
    tau_mean = np.mean(tau_samples, axis=(0, 1)).astype(int)
    change_dates = df.iloc[tau_mean]["Date"]
    
    # Plot results
    fig, ax = plt.subplots(figsize=(15, 5))
    
    # Price series
    ax.plot(df['Date'], df['Price'], label='Brent Oil Price')
    
    # Change points
    for cp in change_dates:
        ax.axvline(cp, color='red', linestyle='--', alpha=0.5)
    
    ax.set_title('Detected Change Points in Brent Oil Prices')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price (USD)')
    plt.savefig('../outputs/figures/change_points.png')
    plt.close()
    
    return change_dates

def quantify_impact(df, change_dates):
    """Quantify impact between change point regimes"""
    results = []
    prev_idx = 0
    
    for i, cp in enumerate(change_dates):
        cp_idx = df[df['Date'] == cp].index[0]
        segment = df.iloc[prev_idx:cp_idx]
        
        if len(segment) > 0:
            results.append({
                'start_date': df.iloc[prev_idx]['Date'],
                'end_date': cp,
                'mean_price': segment['Price'].mean(),
                'volatility': segment['Price'].std(),
                'duration_days': len(segment)
            })
        prev_idx = cp_idx
    
    # Add final segment
    final_segment = df.iloc[prev_idx:]
    results.append({
        'start_date': df.iloc[prev_idx]['Date'],
        'end_date': df.iloc[-1]['Date'],
        'mean_price': final_segment['Price'].mean(),
        'volatility': final_segment['Price'].std(),
        'duration_days': len(final_segment)
    })
    
    return pd.DataFrame(results)

if __name__ == "__main__":
    # Load processed data
    df = pd.read_csv('../data/processed/cleaned_oil_prices.csv', parse_dates=['Date'])
    
    # Run change point detection
    print("Running Bayesian change point analysis...")
    trace = bayesian_change_point_analysis(df)
    
    # Analyze results
    change_dates = analyze_results(trace, df)
    impact_df = quantify_impact(df, change_dates)
    
    # Save results
    impact_df.to_csv('../outputs/change_point_impacts.csv', index=False)
    print("Analysis complete. Results saved to outputs/")

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Running Bayesian change point analysis...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [tau, means, sigma]


c:\Users\gagur\Videos\Change_point_analysis_and_statistical_modeling_of_time_series_data\env\Lib\site-packages\rich
\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

### 2. Event Correlation Analysis

In [ ]:
def correlate_with_events(change_dates, events_file):
    """Correlate detected change points with known events"""
    events = pd.read_csv(events_file, parse_dates=['Date'])
    correlations = []
    
    for cp in change_dates:
        # Find closest event within 30 days
        time_diff = abs(events['Date'] - cp)
        closest_event = events.loc[time_diff.idxmin()]
        
        if time_diff.min() <= pd.Timedelta(days=30):
            correlations.append({
                'change_point_date': cp,
                'event_date': closest_event['Date'],
                'event_name': closest_event['Event'],
                'days_difference': time_diff.min().days
            })
    
    return pd.DataFrame(correlations)

# Example usage after main analysis:
# event_correlations = correlate_with_events(change_dates, '../data/processed/events_annotated.csv')
# event_correlations.to_csv('../outputs/event_correlations.csv', index=False)